In [90]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import re
import joblib
import csv
import os
import codecs

In [91]:
def rename_csv_files(folder_path, title_column, resolve_time_column, sheet_name=None):
    files = os.listdir(folder_path)
    
    for file in files:
         if file.endswith(".csv") and "Shoptab" not in file and "TTS-vi-VN" not in file:
            file_path = os.path.join(folder_path, file)
            if sheet_name is None:
                df = pd.read_csv(file_path)
            else:
                df = pd.read_csv(file_path, sheet_name=sheet_name)
                
            first_title = df[title_column][0]
            
            df[resolve_time_column] = pd.to_datetime(df[resolve_time_column])
            
            start_date = df[resolve_time_column].min()
            end_date = df[resolve_time_column].max()
            
            first_title = re.sub(r'[\\/:"*?<>|]+', '', first_title)
            
            start_date_str = start_date.strftime("%Y-%m-%d")
            end_date_str = end_date.strftime("%Y-%m-%d")
            
            new_file_name = f"{first_title}_{start_date_str}_{end_date_str}.csv"
            new_file_path = os.path.join(folder_path, new_file_name)
            
            if os.path.exists(new_file_path):
                os.remove(new_file_path)
            
            os.rename(file_path, new_file_path)

dir_shoptab = 'D:/Data clean/Shoptab'
dir_UG_ads = 'D:/Data clean/UG 2R GnE'
try:
    rename_csv_files(dir_shoptab, 'title', '1_resolve_time')
    rename_csv_files(dir_UG_ads, 'title', '1_resolve_time')
except: 
    pass

In [92]:
#Supporting Dashboard

#Variable
from datetime import datetime
Date_variable = joblib.load('D:/Code/Joblib_Temp/date_df.pkl')

Start_Date_Shoptab = datetime.strptime(str(Date_variable.loc['sideprj_shoptab', 'Start date']), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_Shoptab = datetime.strptime(str(Date_variable.loc['sideprj_shoptab', 'End date']), '%Y-%m-%d').strftime('%Y-%m-%d')

Start_Date_UGADs = datetime.strptime(str(Date_variable.loc['sideprj_shoptab', 'Start date']), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_UGADs = datetime.strptime(str(Date_variable.loc['sideprj_shoptab', 'Start date']), '%Y-%m-%d').strftime('%Y-%m-%d')

Start_Date_sideproject = datetime.strptime(str(Date_variable.loc['sideprj_dailyreport', 'Start date']), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_sideproject = datetime.strptime(str(Date_variable.loc['sideprj_dailyreport', 'End date']), '%Y-%m-%d').strftime('%Y-%m-%d')

#Queue
queue_list = pd.read_excel('D:/Code/Code/Support_dashboard.xlsx',sheet_name='Queue List').values
queue_list = pd.DataFrame(queue_list)
headers = ['BUSINESS','COMPOUND','CATEGORY','MOD QUEUE ID','MOD QUEUE NAME',
           'QA QUEUE ID','QA QUEUE NAME','QUEUE GROUP','STATUS','Priority','Latency','AHT (secs)','Policy_type','Classify']
queue_list.columns = headers

In [93]:
Date_variable

,Start date,End date
appeal_diff_case,2023-08-13,2023-08-13
appeal_dailyreport,2023-08-12,2023-08-12
regular_diff_case,2023-08-08,2023-08-13
regular_false_case,2023-08-12,2023-08-12
rock_dailyreport,2023-08-12,2023-08-12
PCR_diff_case,2023-08-08,2023-08-13
PCR_false_case,2023-08-12,2023-08-12
sideprj_shoptab,2023-08-13,2023-08-13
sideprj_ugads,2023-08-13,2023-08-13
sideprj_dailyreport,2023-08-12,2023-08-12


In [94]:
#Lấy all data trong folder
def convert_datetime(a):
    my_format="%Y-%m-%d %H:%M:%S"
    try:    
        try:
            return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
        except:
            return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%Y/%m/%d %H:%M:%S").strftime(my_format)
def convert_date(a):
    my_format="%Y-%m-%d"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)
def input_data( data_dir  ):
    list_files = []
        
    for filename in pathlib.Path(data_dir).glob('**/*.csv'):
        df = pd.read_csv(filename, dtype={'object_id':object,
                                          'task_id':object,
                                          'project_id':object})
        df['1_resolve_time'] = df['1_resolve_time'].apply(convert_datetime)
        df['1_resolve_time'] = pd.to_datetime(df['1_resolve_time'])
        df['1_verifier'] = df['1_verifier'].apply(lambda x: re.sub(r'robot_estimate@', '', x))
        list_files.append(df)

    df1 = pd.concat(list_files, ignore_index=True)
    return df1

In [95]:
#Lấy data (cần đổi source)
Shoptab_queue = input_data('D:/Data clean/Shoptab')
UG_ADs_queue = input_data('D:/Data clean/UG 2R GnE')


In [96]:
Shoptab_queue['2_verify_data'][2]

'{"real_assign_time":1690613024.387,"real_resolve_time":1690613086.506,"mainForm":[{"name":"rejectReasons","value":"pass"},{"name":"Label Code","value":null},{"name":"Label Type","value":null},{"name":"Audit Result","value":"Approval"},{"name":"standard_output","value":{"result":200}}],"aht_duration":0,"extra_info":{"video_duration":0,"key_frame_count":0,"contain_private_video":null}}\n'

In [97]:
keymap = {
    'Not approve': ['400', 'notapproval', 'disapproval', 'disapproved', 'violation'],
    'Approve': ['200', 'approval', 'approved', 'general'],
}

def check_string(s):
    if isinstance(s, str):
        s_lower = s.lower()

        for approval_status, substrings in keymap.items():
            if any(substring in s_lower for substring in substrings):
                return approval_status.capitalize()

    return s

Shoptab_queue['1st_result'] = Shoptab_queue['1_audit_result'].apply(check_string)
Shoptab_queue['2nd_result'] = Shoptab_queue['2_audit_result'].apply(check_string)

UG_ADs_queue['1st_result'] = UG_ADs_queue['1_verify_data'].apply(check_string)
UG_ADs_queue['2nd_result'] = UG_ADs_queue['2_verify_data'].apply(check_string)

In [98]:
Shoptab_queue['1st_result'][2]

'Approve'

In [99]:
#Add diff case
Shoptab_queue.loc[Shoptab_queue['1st_result'] != Shoptab_queue['2nd_result'],'Diff case']="FALSE"
Shoptab_queue.loc[Shoptab_queue['Diff case'].isnull(),'Diff case']="TRUE"

UG_ADs_queue.loc[UG_ADs_queue['1st_result'] != UG_ADs_queue['2nd_result'],'Diff case']="FALSE"
UG_ADs_queue.loc[UG_ADs_queue['Diff case'].isnull(),'Diff case']="TRUE"

In [100]:
Shoptab_queue[['task_id', 'project_id','object_id']] = Shoptab_queue[['task_id', 'project_id','object_id']].replace(to_replace=r'id=',value= '',regex=True).astype(str)
UG_ADs_queue[['task_id', 'project_id','object_id']] = UG_ADs_queue[['task_id', 'project_id','object_id']].replace(to_replace=r'id=',value= '',regex=True).astype(str)

In [101]:
Shoptab_queue['2_resolve_time'] = pd.to_datetime(Shoptab_queue['2_resolve_time'])
UG_ADs_queue['2_resolve_time'] = pd.to_datetime(UG_ADs_queue['2_resolve_time'])

Shoptab_queue['1_resolve_time'] = pd.to_datetime(Shoptab_queue['1_resolve_time'])
UG_ADs_queue['1_resolve_time'] = pd.to_datetime(UG_ADs_queue['1_resolve_time'])

Shoptab_queue = Shoptab_queue.rename(columns={'1_label_code':'1_reason',
                                              '2_label_code':'2_reason'})

In [102]:
def filter_time( df,  from_date, to_date):
    Shoptab_queue_filter = Shoptab_queue[(df['2_resolve_time'].dt.strftime('%Y-%m-%d') >= from_date) & (Shoptab_queue['2_resolve_time'].dt.strftime('%Y-%m-%d') <= to_date)]
    Shoptab_queue_filter = Shoptab_queue_filter[['title','project_id','1_verifier','1_resolve_time','object_id','1st_result','1_reason','1_duration','2_verifier','2_resolve_time','task_id','tcs_link','2nd_result','2_reason','Diff case']]
    Shoptab_queue_filter = Shoptab_queue_filter.drop_duplicates(subset='task_id', keep='first')
    Shoptab_queue_filter = pd.DataFrame(Shoptab_queue_filter)
    return Shoptab_queue_filter

final_Shoptab = filter_time(Shoptab_queue, Start_Date_Shoptab,End_Date_Shoptab)

In [103]:
def filter_time( df,  from_date, to_date):
    UG_ADs_queue_filter = UG_ADs_queue[(df['2_resolve_time'].dt.strftime('%Y-%m-%d') >= from_date) & (UG_ADs_queue['2_resolve_time'].dt.strftime('%Y-%m-%d') <= to_date)]
    UG_ADs_queue_filter = UG_ADs_queue_filter[['title','project_id','1_verifier','1_resolve_time','object_id','1st_result','1_reason','1_duration','2_verifier','2_resolve_time','task_id','tcs_link','2nd_result','2_reason','Diff case']]
    UG_ADs_queue_filter = UG_ADs_queue_filter.drop_duplicates(subset='task_id', keep='first')
    UG_ADs_queue_filter = pd.DataFrame(UG_ADs_queue_filter)
    return UG_ADs_queue_filter

final_UG_ADs = filter_time(UG_ADs_queue, Start_Date_UGADs,End_Date_UGADs)

In [104]:
side_project_append = pd.concat([Shoptab_queue, UG_ADs_queue], axis=0, ignore_index=True)
side_project_append.head(1)

,task_id,project_id,title,object_id,status,create_time,closed_time,audit_status,1_verifier,1_assign_time,...,directly_submit_label_type,directly_submit_rejectReasons,directly_submit_standard_output,1st_result,2nd_result,Diff case,1_rate,1_tcs_audit_status,2_rate,2_tcs_audit_status
0,7261005744772792838,7260106686818451970,QA [CB][VN]-Shoptab Product Image Dedicated,tos-maliva-i-o3syd03w52-us/138e1e8a1977433d8db...,Closed,2023-07-29 05:50:41,2023-07-29 13:44:47,none,duong.tla@trans-cosmos.com.vn,2023-07-29 05:50:41,...,NaN,NaN,NaN,Approve,Approve,TRUE,NaN,NaN,NaN,NaN


In [105]:
def filter_time( df,  from_date, to_date):
    side_project_diff_temp = side_project_append[(df['1_resolve_time'].dt.strftime('%Y-%m-%d') >= from_date) & (side_project_append['1_resolve_time'].dt.strftime('%Y-%m-%d') <= to_date)]
    side_project_diff_temp = side_project_diff_temp[['title','project_id','1_verifier','1_resolve_time','object_id','1st_result','1_reason','1_duration','2_verifier','2_resolve_time','task_id','tcs_link','2nd_result','2_reason','Diff case']]
    side_project_diff_temp = pd.DataFrame(side_project_diff_temp)
    side_project_diff_temp[['1_reason','2_reason']] = side_project_diff_temp[['1_reason','2_reason']].replace(to_replace='null',value= '',regex=True).astype(str)

    return side_project_diff_temp

side_project_diff_case = filter_time(side_project_append, Start_Date_Shoptab, End_Date_Shoptab)


In [106]:
side_project_append.columns

Index(['task_id', 'project_id', 'title', 'object_id', 'status', 'create_time',
       'closed_time', 'audit_status', '1_verifier', '1_assign_time',
       '1_resolve_time', '1_duration', '1_verify_data', '1_audit_result',
       '1_reason', '1_label_type', '1_rejectReasons', '1_standard_output',
       '2_verifier', '2_assign_time', '2_resolve_time', '2_duration',
       '2_verify_data', '2_audit_result', '2_reason', '2_label_type',
       '2_rejectReasons', '2_standard_output', 'tcs_link', 'is_simulation',
       'Diff', 'directly_submit_verifier', 'directly_submit_assign_time',
       'directly_submit_resolve_time', 'directly_submit_duration',
       'directly_submit_verify_data', 'directly_submit_audit_result',
       'directly_submit_label_code', 'directly_submit_label_type',
       'directly_submit_rejectReasons', 'directly_submit_standard_output',
       '1st_result', '2nd_result', 'Diff case', '1_rate', '1_tcs_audit_status',
       '2_rate', '2_tcs_audit_status'],
      dtype='o

In [107]:
def filter_time( df,  from_date, to_date):
    side_project_temp = side_project_append[(df['1_resolve_time'].dt.strftime('%Y-%m-%d') >= from_date) & (side_project_append['1_resolve_time'].dt.strftime('%Y-%m-%d') <= to_date)]
    side_project_temp = side_project_temp[['title','project_id','1_verifier','1_resolve_time','object_id','1st_result','1_reason','1_duration','2_verifier','2_resolve_time','task_id','tcs_link','2nd_result','2_reason','Diff case']]
    side_project_temp = pd.DataFrame(side_project_temp)
    side_project_temp = side_project_temp.drop_duplicates(subset='task_id', keep='first')

    return side_project_temp

side_project_temp = filter_time(side_project_append, Start_Date_sideproject, End_Date_sideproject)

side_project_temp['1_resolve_time'] = pd.to_datetime(side_project_temp['1_resolve_time'].apply(convert_date))

In [108]:
#Sample size - side project

side_project_sample = side_project_temp.groupby(by=['1_resolve_time','title','1_verifier'], as_index=False).agg({'Diff case':'count'}).rename(columns={'1_resolve_time':'Moderation time',
                                                                                                                                                       'title':'Sampling Queue',
                                                                                                                                                       '1_verifier':'Moderator name',
                                                                                                                                                       'Diff case':'No. of Samples'})

In [109]:
#Diff case - side project

side_project_false = side_project_temp[side_project_temp['Diff case']=="FALSE"]
side_project_false = side_project_false.groupby(by=['1_resolve_time','title','1_verifier'], as_index=False).agg({'Diff case':'count'}).rename(columns={'1_resolve_time':'Moderation time',
                                                                                                                                                       'title':'Sampling Queue',
                                                                                                                                                       '1_verifier':'Moderator name',
                                                                                                                                                       'Diff case':'No. Diff cases'})

In [110]:
side_project_false

,Moderation time,Sampling Queue,Moderator name,No. Diff cases


In [111]:
#Daily report - side project
daily_side_performance = pd.merge(side_project_sample,side_project_false,how='left',left_on=['Moderation time','Sampling Queue','Moderator name'],right_on=['Moderation time','Sampling Queue','Moderator name']).fillna(0)

In [112]:
dr_side_groupqueue = pd.merge(daily_side_performance,queue_list,how='left',left_on=['Sampling Queue'],right_on=['QA QUEUE NAME'])

In [113]:
#Daily report - side project group queue
dr_side_groupqueue = pd.merge(daily_side_performance,queue_list,how='left',left_on=['Sampling Queue'],right_on=['QA QUEUE NAME'])
dr_side_groupqueue = dr_side_groupqueue[['Moderation time','MOD QUEUE NAME','No. of Samples','No. Diff cases']]
dr_side_groupqueue = dr_side_groupqueue.groupby(['Moderation time','MOD QUEUE NAME'],as_index=False).agg({'No. of Samples':'sum','No. Diff cases':'sum'})
dr_side_groupqueue['%Accr'] = round(1-(dr_side_groupqueue['No. Diff cases'] / dr_side_groupqueue['No. of Samples']),4)
dr_side_groupqueue

,Moderation time,MOD QUEUE NAME,No. of Samples,No. Diff cases,%Accr
0,2023-08-12,[CB][VN]-Shoptab Product Image Dedicated,57,0.0,1.0
1,2023-08-12,[VN] Shoptab Content cover image,122,0.0,1.0


In [114]:
all_sheets_sideproject = {'Diff_case_sideproject':side_project_diff_case,
                        'Daily_Performance_sideproject':daily_side_performance,
                        'Sample_size_sideproject':side_project_sample,
                        'False_case_sideproject':side_project_false,
                        'DR_group_queue_sideproject': dr_side_groupqueue}

joblib.dump(all_sheets_sideproject,'D:/Code/Joblib_Temp/side project temp.pkl')

['D:/Code/Joblib_Temp/side project temp.pkl']

In [115]:
Regular_writer = pd.ExcelWriter('C:/Users/v6210227/Desktop/side project.xlsx', engine='xlsxwriter')

for sheet_name in all_sheets_sideproject.keys():
    all_sheets_sideproject[sheet_name].to_excel(excel_writer=Regular_writer, sheet_name=sheet_name)
   
Regular_writer.close()

In [116]:
#Code test